In [35]:
from rdkit import Chem
import numpy as np
import pandas as pd
import os

from search_uaa import contains_uaa

In [36]:
inclusiveDB = pd.read_csv("uaa_data/inclusive_db.csv", sep=";")
inclusiveDB.tail()

,ID,ncAA abbreviation(s) used in the publication,"ncAA name, as mentioned in the publication",ncAA IUPAC name,ncAA SMILES notation,ncAA chemical formula,ncAA Structure,"ncAA link for Pubchem, if available","Application, if provided in the publication",Canonical amino acid most similar to the ncAA,...,tRNA organism,tRNA sequence,Codon suppression,Tested in (protein),Tested in (protein position),Tested in (organism/in vitro),Comment (if applicable),Publication,DOI,Publication year
2427,2428,DHFW,"6,7-dihydrofuran-Trp","(2S)-2-amino-3-(7,8-dihydro-1H-furo[2,3-g]indo...",C1COC2=C1C3=C(C=C2)C(=CN3)CC(C(=O)O)N,C13H14N2O3,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/1684...,manipulating cation−π Interactions,Tryptophan,...,Homo sapiens + Methanosarcina barkeri + Methan...,GCCGAGATGATCATGTAGATCGAACGGACTCTAAATCCGTTCAGCC...,stop codon suppression TAG (Amber),Homo sapiens H3K4me3 reader domain PHD of lysi...,W28,E coli DH10B,chimeric synthetase is created by using MbPyl...,"Zhao, H., Tang, L., Fang, Y., Liu, C., Ding, W...",https://doi.org/10.1021/jacs.3c02293,2023
2428,2429,6MeOW,6-methoxy-Trp,(S)-2-Amino-3-(6-methoxy-1H-indol-3-yl)propano...,COC1=CC2=C(C=C1)C(=CN2)CC(C(=O)O)N,C12H14N2O3,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/1540...,manipulating cation−π Interactions,Tryptophan,...,Homo sapiens + Methanosarcina barkeri + Methan...,GCCGAGATGATCATGTAGATCGAACGGACTCTAAATCCGTTCAGCC...,stop codon suppression TAG (Amber),EGFP,*190,E coli DH10B,chimeric synthetase is created by using MbPyl...,"Zhao, H., Tang, L., Fang, Y., Liu, C., Ding, W...",https://doi.org/10.1021/jacs.3c02293,2023
2429,2430,DiMeW,"6,7-dimethyl-Trp","(2S)-2-amino-3-(6,7-dimethyl-1H-indol-3-yl)pro...",CC1=C(C2=C(C=C1)C(=CN2)CC(C(=O)O)N)C,C13H16N2O2,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/9680...,manipulating cation−π Interactions,Tryptophan,...,Homo sapiens + Methanosarcina barkeri + Methan...,GCCGAGATGATCATGTAGATCGAACGGACTCTAAATCCGTTCAGCC...,stop codon suppression TAG (Amber),EGFP,*190,E coli DH10B,chimeric synthetase is created by using MbPyl...,"Zhao, H., Tang, L., Fang, Y., Liu, C., Ding, W...",https://doi.org/10.1021/jacs.3c02293,2023
2430,2431,DiMeOW,"6,7-dimethoxy-Trp","(2S)-2-amino-3-(6,7-dimethoxy-1H-indol-3-yl)pr...",COC1=C(C2=C(C=C1)C(=CN2)CC(C(=O)O)N)OC,C13H16N2O4,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/1684...,manipulating cation−π Interactions,Tryptophan,...,Homo sapiens + Methanosarcina barkeri + Methan...,GCCGAGATGATCATGTAGATCGAACGGACTCTAAATCCGTTCAGCC...,stop codon suppression TAG (Amber),EGFP,*190,E coli DH10B,chimeric synthetase is created by using MbPyl...,"Zhao, H., Tang, L., Fang, Y., Liu, C., Ding, W...",https://doi.org/10.1021/jacs.3c02293,2023
2431,2432,DHFW,"6,7-dihydrofuran-Trp","(2S)-2-amino-3-(7,8-dihydro-1H-furo[2,3-g]indo...",C1COC2=C1C3=C(C=C2)C(=CN3)CC(C(=O)O)N,C13H14N2O3,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/1684...,manipulating cation−π Interactions,Tryptophan,...,Homo sapiens + Methanosarcina barkeri + Methan...,GCCGAGATGATCATGTAGATCGAACGGACTCTAAATCCGTTCAGCC...,stop codon suppression TAG (Amber),EGFP,*190,E coli DH10B,chimeric synthetase is created by using MbPyl...,"Zhao, H., Tang, L., Fang, Y., Liu, C., Ding, W...",https://doi.org/10.1021/jacs.3c02293,2023


In [37]:

UNNATURAL_RESIDUES = inclusiveDB[['ncAA abbreviation(s) used in the publication', 'ncAA SMILES notation']]
UNNATURAL_RESIDUES = UNNATURAL_RESIDUES[~UNNATURAL_RESIDUES.apply(lambda row: row.astype(str).str.contains("not available", case=False)).any(axis=1)]

#UNNATURAL_RESIDUES = dict(zip(UNNATURAL_RESIDUES.iloc[:, 0], UNNATURAL_RESIDUES.iloc[:, 1]))
UNNATURAL_RESIDUES

,ncAA abbreviation(s) used in the publication,ncAA SMILES notation
1,4fW,C1=CC2=C(C(=C1)F)C(=CN2)CC(C(=O)O)N
2,pAzPhe,C1=CC(=CC=C1CC(C(=O)O)N)N=[N+]=[N-]
3,pAzPhe,C1=CC(=CC=C1CC(C(=O)O)N)N=[N+]=[N-]
4,pAzPhe,C1=CC(=CC=C1CC(C(=O)O)N)N=[N+]=[N-]
5,pBpa,C1=CC=C(C=C1)C(=O)C2=CC=C(C=C2)CC(C(=O)O)N
...,...,...
2427,DHFW,C1COC2=C1C3=C(C=C2)C(=CN3)CC(C(=O)O)N
2428,6MeOW,COC1=CC2=C(C=C1)C(=CN2)CC(C(=O)O)N
2429,DiMeW,CC1=C(C2=C(C=C1)C(=CN2)CC(C(=O)O)N)C
2430,DiMeOW,COC1=C(C2=C(C=C1)C(=CN2)CC(C(=O)O)N)OC


In [39]:
import os
import pandas as pd

SSC_SMI_DIR = "/Users/sanjitrao/Downloads/L_SMI"


def fix_scc_smiles(s):
    s = s.replace("[NH3]", "[NH3+]")
    s = s.replace("[Cl](=O)=O", "ClS(=O)=O")  # example fix; adjust as needed
    return s

def load_ssc_smiles(smidir: str):
    rows = []
    for fname in os.listdir(smidir):
        if fname.endswith(".smi"):
            path = os.path.join(smidir, fname)
            with open(path, "r") as f:
                line = f.read().strip()

                if not line:
                    continue

                # typical format: "<smiles> <code>"
                parts = line.split()
                smiles = parts[0]
                code = parts[-1]  # 3-letter / extended code ("YCM", "WFP", etc.)

                rows.append((code, smiles))

    data =  pd.DataFrame(rows, columns=["ssc_code", "ssc_smiles"])
    data['ssc_code'] = data['ssc_code'].str.replace('.pdb', '', regex=False)


    data['ssc_smiles'] = data['ssc_smiles'].apply(fix_scc_smiles)
    return data


SSC = load_ssc_smiles(SSC_SMI_DIR)
SSC.head()


,ssc_code,ssc_smiles
0,ALN,O=[C](=O)[C@H](Cc1cccc2c1cccc2)[NH3+]
1,OCY,OCCSC[C@@H]([C](=O)=O)[NH3+]
2,0AF,[NH3+][C@H]([C](=O)=O)Cc1c[nH]c2c1cccc2O
3,26P,OC(=O)[C@H](CCC[C](=[C](=O)=O)=O)[NH3+]
4,BHD,O=[C](=O)[C@H]([C@@H]([C](=O)=O)[NH3+])O


In [ ]:
from rdkit import Chem

def canon(x):
    mol = Chem.MolFromSmiles(x)
    if mol is None:
        return None
    return Chem.MolToSmiles(mol, canonical=True)

UNNATURAL_RESIDUES["canon"] = UNNATURAL_RESIDUES["ncAA SMILES notation"].apply(canon)
SSC["canon"] = SSC["ssc_smiles"].apply(canon)

In [41]:
UNNATURAL_RESIDUES

,ncAA abbreviation(s) used in the publication,ncAA SMILES notation,canon
1,4fW,C1=CC2=C(C(=C1)F)C(=CN2)CC(C(=O)O)N,NC(Cc1c[nH]c2cccc(F)c12)C(=O)O
2,pAzPhe,C1=CC(=CC=C1CC(C(=O)O)N)N=[N+]=[N-],[N-]=[N+]=Nc1ccc(CC(N)C(=O)O)cc1
3,pAzPhe,C1=CC(=CC=C1CC(C(=O)O)N)N=[N+]=[N-],[N-]=[N+]=Nc1ccc(CC(N)C(=O)O)cc1
4,pAzPhe,C1=CC(=CC=C1CC(C(=O)O)N)N=[N+]=[N-],[N-]=[N+]=Nc1ccc(CC(N)C(=O)O)cc1
5,pBpa,C1=CC=C(C=C1)C(=O)C2=CC=C(C=C2)CC(C(=O)O)N,NC(Cc1ccc(C(=O)c2ccccc2)cc1)C(=O)O
...,...,...,...
2427,DHFW,C1COC2=C1C3=C(C=C2)C(=CN3)CC(C(=O)O)N,NC(Cc1c[nH]c2c3c(ccc12)OCC3)C(=O)O
2428,6MeOW,COC1=CC2=C(C=C1)C(=CN2)CC(C(=O)O)N,COc1ccc2c(CC(N)C(=O)O)c[nH]c2c1
2429,DiMeW,CC1=C(C2=C(C=C1)C(=CN2)CC(C(=O)O)N)C,Cc1ccc2c(CC(N)C(=O)O)c[nH]c2c1C
2430,DiMeOW,COC1=C(C2=C(C=C1)C(=CN2)CC(C(=O)O)N)OC,COc1ccc2c(CC(N)C(=O)O)c[nH]c2c1OC


In [42]:
SSC

,ssc_code,ssc_smiles,canon
0,ALN,O=[C](=O)[C@H](Cc1cccc2c1cccc2)[NH3+],None
1,OCY,OCCSC[C@@H]([C](=O)=O)[NH3+],None
2,0AF,[NH3+][C@H]([C](=O)=O)Cc1c[nH]c2c1cccc2O,None
3,26P,OC(=O)[C@H](CCC[C](=[C](=O)=O)=O)[NH3+],None
4,BHD,O=[C](=O)[C@H]([C@@H]([C](=O)=O)[NH3+])O,None
...,...,...,...
225,MOT5,COc1ccc2c(c1)c(c[nH]2)C[C@@H](C(=O)O)[NH3+],COc1ccc2[nH]cc(C[C@H]([NH3+])C(=O)O)c2c1
226,PYR4,[NH3+][C@H](C(=O)O)Cc1ccncc1,[NH3+][C@@H](Cc1ccncc1)C(=O)O
227,QX32,OC(=O)[C@H](Cc1cnc2c(n1)cccc2)[NH3+],[NH3+][C@@H](Cc1cnc2ccccc2n1)C(=O)O
228,ALC,[NH3+][C@H](C(=O)O)CC1CCCCC1,[NH3+][C@@H](CC1CCCCC1)C(=O)O


In [46]:
SSC_noNA = SSC['canon'].dropna()
MERGED = UNNATURAL_RESIDUES.merge(SSC_noNA, on="canon", how="inner")
MERGED

,ncAA abbreviation(s) used in the publication,ncAA SMILES notation,canon
